<a href="https://colab.research.google.com/github/Lemian5/DataSet/blob/main/CasoReal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import pairwise_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import random

# Crear un conjunto de datos de ejemplo para productos
productos_data = {
    'ID': [1, 2, 3, 4, 5],
    'Nombre': ['Producto A', 'Producto B', 'Producto C', 'Producto D', 'Producto E'],
    'Descripción': ['Descripción A', 'Descripción B', 'Descripción C', 'Descripción D', 'Descripción E']
}

productos_df = pd.DataFrame(productos_data)

# Crear un conjunto de datos de ejemplo para las interacciones de los usuarios
interacciones_data = {
    'UsuarioID': [1, 1, 2, 2, 3, 4],
    'ProductoID': [1, 2, 2, 3, 4, 5],
    'Calificación': [5, 4, 5, 3, 4, 5]
}

interacciones_df = pd.DataFrame(interacciones_data)

# Sistema de Recomendación basado en Contenido
def sistema_recomendacion_contenido(productos_df, interacciones_df, usuario_id):
    # Fusionar los datos de productos e interacciones
    datos_fusionados = pd.merge(interacciones_df, productos_df, left_on='ProductoID', right_on='ID')

    # Crear una matriz de términos del documento (TF-IDF) para las descripciones de los productos
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(productos_df['Descripción'])

    # Calcular la similitud de coseno entre las descripciones de los productos
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

    # Obtener las interacciones del usuario
    interacciones_usuario = datos_fusionados[datos_fusionados['UsuarioID'] == usuario_id]

    # Crear una lista de productos ya interactuados por el usuario
    productos_interactuados = list(interacciones_usuario['ProductoID'])

    # Calcular las puntuaciones de similitud de coseno entre el producto interactuado y todos los productos
    sim_scores = list(enumerate(cosine_sim[productos_interactuados[-1]]))

    # Ordenar los productos según la puntuación de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Obtener los productos recomendados excluyendo los ya interactuados
    productos_recomendados = [item[0] for item in sim_scores if item[0] not in productos_interactuados]

    return productos_recomendados[:3]  # Devolver los tres primeros productos recomendados

# Chatbot de Atención al Cliente
def chatbot_atencion_cliente():
    respuestas = {
        'saludo': '¡Hola! ¿En qué puedo ayudarte hoy?',
        'despedida': 'Gracias por visitarnos. ¡Que tengas un buen día!',
        'ayuda': 'Puedo ayudarte con preguntas sobre productos, envíos y más. ¿En qué necesitas asistencia?',
        'default': 'Lo siento, no entendí tu pregunta. ¿Puedes ser más específico?'
    }

    while True:
        usuario_pregunta = input('Usuario: ')

        if 'hola' in usuario_pregunta.lower():
            print('Chatbot:', respuestas['saludo'])
        elif 'gracias' in usuario_pregunta.lower():
            print('Chatbot:', respuestas['despedida'])
            break
        elif 'ayuda' in usuario_pregunta.lower():
            print('Chatbot:', respuestas['ayuda'])
        else:
            print('Chatbot:', respuestas['default'])

# Ejemplo de uso del sistema de recomendación y chatbot
usuario_id_ejemplo = 1

# Recomendación de productos para el usuario
productos_recomendados = sistema_recomendacion_contenido(productos_df, interacciones_df, usuario_id_ejemplo)
print(f"Productos recomendados para el Usuario {usuario_id_ejemplo}:")

for producto_id in productos_recomendados:
    # Verificar si el producto_id está presente en el conjunto de datos
    if producto_id not in productos_df['ID'].values:
        print(f"¡Error! El producto con ID {producto_id} no está presente en el conjunto de datos.")
    else:
        # Obtener el nombre del producto correspondiente al ID
        nombre_producto = productos_df.loc[productos_df['ID'] == producto_id, 'Nombre'].values
        if len(nombre_producto) > 0:
            print(nombre_producto[0])
        else:
            print(f"¡Error! No se pudo obtener el nombre del producto con ID {producto_id}.")


Productos recomendados para el Usuario 1:
¡Error! El producto con ID 0 no está presente en el conjunto de datos.
Producto C
Producto D
